# Heat Waves and Cold Spells from Climate Projections

Der Datensatz enthält die Anzahl der Hitzewellen-Tage, die mit verschiedenen europaweiten sowie nationalen/regionalen Definitionen im Rahmen des C3S European Health Service entwickelt wurden. Als Basis dienten Modelldaten des EURO-CORDEX Ensembles, wobei die Hitzewellen-Tage für unterschiedliche zukünftige Zeiträume und Klimawandelszenarien verfügbar sind.

> **Informationen zum Datensatz**: 
>
> https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-heat-and-cold-spells?tab=overview
>
> **Author:** T. Tewes (Stadt Konstanz) 
>
> **Notebook-Version:** 1.0 (18.07.2024)


### 1. Festlegen der Verzeichnisse und Variablen

In [5]:
## 1. Festlegen der Verzeichnisse und Variablen

zip_file = "HWD.zip" ## <- Kann angepasst werden: Name, unter dem die ZIP-File gespeichert wird
download_folder = r"C:\Users\timte\Documents\Python\HWD\Download" ## <- Bitte anpassen: Pfad, unter dem die ZIP-Datei heruntergeladen und gespeichert werden soll 
working_folder = r"C:\Users\timte\Documents\Python\HWD" ## <-- Bitte anpassen: Ordner, in dem die ZIP-Datei extrahiert und die netCDFs gespeichert werden sollen
output_folder = r"\Users\timte\Documents\Python\HWD\results" ## <-- Bitte anpassen: Ordner, in dem die Ergebnisse (Layer, Tabellen) der Datenprozessierung gespeichert werden sollen

import os

# Falls die Ordner noch nicht existieren, diese erstellen
if not os.path.exists(working_folder):
    os.makedirs(working_folder)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

download_path = os.path.join(download_folder, zip_file)
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

### 2. Herunterladen des Datensatzes + Entpacken

In [3]:
## 2.1 Download des Datensatzes über die CDS-API


"""
Bitte unter api_key Ihren API-Key angeben    
"""
api_key = "000000:11111111-2222-3333-4444-555566667777" ## <-Bitte hier den API-Key einfügen 


import cdsapi

# Erstellt einen CDS-API-Client
c = cdsapi.Client(url='https://cds.climate.copernicus.eu/api/v2', key=api_key)


# Download des Datensatzes
c.retrieve(
    'sis-heat-and-cold-spells',
    {
        'variable': 'heat_wave_days',
        'definition': 'climatological_related',
        'experiment': [
            'rcp4_5', 'rcp8_5',
        ],
        'ensemble_statistic': [
            'ensemble_members_average', 'ensemble_members_standard_deviation',
        ],
        'format': 'zip',
    },
    download_path)


print(f'Datei {zip_file} wurde heruntergeladen und in {download_folder} gespeichert')

2024-07-18 08:51:02,026 INFO Welcome to the CDS
2024-07-18 08:51:02,026 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-heat-and-cold-spells
2024-07-18 08:51:02,333 INFO Request is queued
2024-07-18 08:51:03,367 INFO Request is running
2024-07-18 08:51:23,344 INFO Request is completed
2024-07-18 08:51:23,344 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data7/dataset-sis-heat-and-cold-spells-305efce6-43b6-459d-92ce-a0f669ad7d85.zip to C:\Users\timte\Documents\Python\HWD\Download\HWD.zip (162.8M)
2024-07-18 08:51:46,909 INFO Download rate 6.9M/s  


Datei HWD.zip wurde heruntergeladen und in C:\Users\timte\Documents\Python\HWD\Download gespeichert


In [4]:
## 2.2 Entpacken der heruntergeladenenen Zip-Datei

import zipfile
import os

with zipfile.ZipFile(download_path, 'r') as zip_ref:
    zip_ref.extractall(working_folder)
    
print(f"ZIP-Datei wurde in das Verzeichnis {working_folder} entpackt")

ZIP-Datei wurde in das Verzeichnis C:\Users\timte\Documents\Python\HWD entpackt


### 3. Auswahl der netCDF-Datei / Festlegen des RCP-Szenarios und der Statistik

In [6]:
## 3. Auswahl der netCDF-Datei anhand Benutzereingabe

import os
import re

def list_nc_files(folder):
    """
    List all .nc files in the given folder.
    """
    return [f for f in os.listdir(folder) if f.endswith('.nc')]

def select_file(files):
    """
    Prompt the user to select a file from the list by entering a number.
    """
    for idx, file in enumerate(files):
        print(f"{idx + 1}: {file}")
    
    while True:
        try:
            choice = int(input("Wählen Sie eine Datei durch Eingabe der entsprechenden Zahl: "))
            if 1 <= choice <= len(files):
                return files[choice - 1]
            else:
                print("Ungültige Wahl. Bitte geben Sie eine Zahl zwischen 1 und", len(files), "ein.")
        except ValueError:
            print("Ungültige Eingabe. Bitte geben Sie eine Zahl ein.")


# Alle .nc-Dateien in dem definierten Ordner auflisten
nc_files = list_nc_files(working_folder)

# Benutzeraufforderung, eine Datei auszuwählen
selected_file = select_file(nc_files)
input_netCDF = os.path.join(working_folder, selected_file)

match = re.search(r'rcp(\d{2})_(\w+)_v', selected_file)
if match:
    rcp = f'rcp{match.group(1)}'
    statistic = match.group(2)
    
    print(f"Dateiname: {selected_file}")
    print(f"rcp: {rcp}")
    print(f"statistic: {statistic}")
else:
    print(f"Kein passender Teil im Dateinamen: {selected_file}")

# Gibt die ausgewählte Datei wieder
print("")
print(f"Folgende Datei wurde ausgewählt: {input_netCDF}")


1: HWD_EU_climate_rcp45_mean_v1.0.nc
2: HWD_EU_climate_rcp45_stdev_v1.0.nc
3: HWD_EU_climate_rcp85_mean_v1.0.nc
4: HWD_EU_climate_rcp85_stdev_v1.0.nc


Wählen Sie eine Datei durch Eingabe der entsprechenden Zahl:  1


Dateiname: HWD_EU_climate_rcp45_mean_v1.0.nc
rcp: rcp45
statistic: mean

Folgende Datei wurde ausgewählt: C:\Users\timte\Documents\Python\HWD\HWD_EU_climate_rcp45_mean_v1.0.nc


Optional: Betrachten der ausgewählten netCDF über xarray

In [ ]:
import xarray as xr
nc_file = xr.open_dataset(input_netCDF)
nc_file

### 4. Konvertieren aller Zeitschritte in GeoTIFFs

In [11]:
## 4. Konvertieren aller Zeitschritte in GeoTIFFS

import os
import xarray as xr
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
import netCDF4 as nc
import numpy as np
import shutil


# NetCDF-Datei öffnen
ds = nc.Dataset(input_netCDF)
lat = ds.variables['lat'][:]
lon = ds.variables['lon'][:]
time_var = ds.variables['time']
time_units = ds.variables['time'].units  # Zeit-Einheiten ablesen

# Bestimme die Pixelgröße und den Ursprung
pixel_size_lat = (lat.max() - lat.min()) / len(lat)
pixel_size_lon = (lon.max() - lon.min()) / len(lon)
transform = from_origin(west=lon.min(), north=lat.max(), xsize=pixel_size_lon, ysize=pixel_size_lat)

# Definiere das CRS
proj_string = "+proj=ob_tran +o_proj=longlat +o_lon_p=0 +o_lat_p=39.25 +lon_0=18 +to_meter=0.01745329"
crs = CRS.from_proj4(proj_string)

# Unterordner für unprozessierte GeoTIFFs erstellen
unprocessed_dir = os.path.join(output_folder, 'unprocessed')
os.makedirs(unprocessed_dir, exist_ok=True)

print(f"Pixel Size Lat {pixel_size_lat}")
print(f"Pixel Size Lon {pixel_size_lon}")

Pixel Size Lat 0.09976470588235294
Pixel Size Lon 0.0998330550918197


In [15]:
import os
import shutil
import rasterio
import numpy as np
from netCDF4 import Dataset, num2date

try:
    # Öffnen der NetCDF-Datei
    ds = Dataset(input_netCDF, 'r')
    time_var = ds.variables['time']
    time_units = ds.variables['time'].units

    # Schleife über alle Zeitdaten
    for t in range(len(time_var)):
        variable_name = 'HWD_EU_climate'
        # Daten für den aktuellen Zeitstempel lesen
        data = ds.variables[variable_name][t, :, :]

        # Jahr aus der Zeitvariable extrahieren
        year = int(num2date(time_var[t], time_units).year)

        # Speichern als unprozessierter GeoTIFF
        geotiff_path = os.path.join(unprocessed_dir, f'{variable_name}_{rcp}_{statistic}_{year}.tiff')
        with rasterio.open(
            geotiff_path,
            'w',
            driver='GTiff',
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=crs,
            transform=transform
        ) as dst:
            dst.write(data, 1)

        # Bild um 180° drehen und von links nach rechts spiegeln
        rotated_data = np.rot90(data, k=2)
        flipped_data = np.fliplr(rotated_data)

        # Speichern des bearbeiteten GeoTIFFs
        processed_path = os.path.join(output_folder, f'{variable_name}_{rcp}_{statistic}_{year}_processed.tiff')
        with rasterio.open(
            processed_path,
            'w',
            driver='GTiff',
            height=flipped_data.shape[0],
            width=flipped_data.shape[1],
            count=1,
            dtype=flipped_data.dtype,
            crs=crs,
            transform=transform
        ) as dst:
            dst.write(flipped_data, 1)

        print(f"GeoTIFF für Jahr {year} gespeichert: '{geotiff_path}' und bearbeitet als '{processed_path}'")

    # Datei schließen
    ds.close()

    # Löschen des unprozessierten Ordners und seiner Inhalte
    shutil.rmtree(unprocessed_dir)
    print(f"Der Ordner '{unprocessed_dir}' und seine Inhalte wurden gelöscht.")

except Exception as e:
    print(f"Fehler beim Öffnen der Datei {input_netCDF}: {e}")


GeoTIFF für Jahr 1986 gespeichert: '\Users\timte\Documents\Python\HWD\results\unprocessed\HWD_EU_climate_rcp45_mean_1986.tiff' und bearbeitet als '\Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_mean_1986_processed.tiff'
GeoTIFF für Jahr 1987 gespeichert: '\Users\timte\Documents\Python\HWD\results\unprocessed\HWD_EU_climate_rcp45_mean_1987.tiff' und bearbeitet als '\Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_mean_1987_processed.tiff'
GeoTIFF für Jahr 1988 gespeichert: '\Users\timte\Documents\Python\HWD\results\unprocessed\HWD_EU_climate_rcp45_mean_1988.tiff' und bearbeitet als '\Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_mean_1988_processed.tiff'
GeoTIFF für Jahr 1989 gespeichert: '\Users\timte\Documents\Python\HWD\results\unprocessed\HWD_EU_climate_rcp45_mean_1989.tiff' und bearbeitet als '\Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_mean_1989_processed.tiff'
GeoTIFF für Jahr 1990 gespeichert: '\Users\timte\Documen

### 5. Exportieren als CSV

In [9]:
import os
import pandas as pd
from netCDF4 import Dataset, num2date

# Funktion zum Umwandeln von netCDF in CSV
def convert_netcdf_to_csv(nc_file, csv_file):
    # Öffne die netCDF Datei
    dataset = Dataset(nc_file, 'r')
    
    # Extrahiere die spezifische Variable und die Zeit
    if 'HWD_EU_climate' in dataset.variables and 'time' in dataset.variables:
        temperature = dataset.variables['HWD_EU_climate'][:]
        time = dataset.variables['time'][:]
        lat = dataset.variables['lat'][:]
        lon = dataset.variables['lon'][:]
        
        # Konvertiere die Zeit in ein lesbares Format
        time_units = dataset.variables['time'].units
        time_calendar = dataset.variables['time'].calendar if hasattr(dataset.variables['time'], 'calendar') else 'standard'
        time = num2date(time, units=time_units, calendar=time_calendar)
        
        # Definiere die Grenzen für Deutschland
        lat_min, lat_max = 47, 55
        lon_min, lon_max = 5, 15
        
        # Finde die Indizes, die in den Bereich von Deutschland fallen
        lat_indices = (lat >= lat_min) & (lat <= lat_max)
        lon_indices = (lon >= lon_min) & (lon <= lon_max)
        
        # Filtere die Daten für Deutschland
        filtered_lat = lat[lat_indices]
        filtered_lon = lon[lon_indices]
        filtered_temperature = temperature[:, lat_indices, :][:, :, lon_indices]
        
        # Erstelle eine Liste von Einträgen für das DataFrame
        rows = []
        for t in range(filtered_temperature.shape[0]):
            for i in range(filtered_temperature.shape[1]):
                for j in range(filtered_temperature.shape[2]):
                    rows.append({
                        'time': time[t],
                        'latitude': filtered_lat[i],
                        'longitude': filtered_lon[j],
                        'HWD_EU_climate': filtered_temperature[t, i, j]
                    })
        
        # Erstelle ein DataFrame aus den Einträgen
        df = pd.DataFrame(rows)
        
        # Speichere das DataFrame als CSV Datei
        df.to_csv(csv_file, index=False)
        
        print(f'Converted: {nc_file} to {csv_file}')
    else:
        print(f'Variables not found in {nc_file}')
    
    # Schließe das Dataset
    dataset.close()

# Erstelle das Ausgabe-Verzeichnis, falls es nicht existiert
os.makedirs(output_folder, exist_ok=True)

# Durchlaufe alle Dateien im Verzeichnis
for filename in os.listdir(working_folder):
    if filename.endswith('.nc'):
        nc_file = os.path.join(working_folder, filename)
        csv_file = os.path.join(output_folder, filename.replace('.nc', '.csv'))
        
        # Konvertiere die Datei
        convert_netcdf_to_csv(nc_file, csv_file)


Converted: C:\Users\timte\Documents\Python\HWD\HWD_EU_climate_rcp45_mean_v1.0.nc to \Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_mean_v1.0.csv
Converted: C:\Users\timte\Documents\Python\HWD\HWD_EU_climate_rcp45_stdev_v1.0.nc to \Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp45_stdev_v1.0.csv
Converted: C:\Users\timte\Documents\Python\HWD\HWD_EU_climate_rcp85_mean_v1.0.nc to \Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp85_mean_v1.0.csv
Converted: C:\Users\timte\Documents\Python\HWD\HWD_EU_climate_rcp85_stdev_v1.0.nc to \Users\timte\Documents\Python\HWD\results\HWD_EU_climate_rcp85_stdev_v1.0.csv
